# Web-Scraping-Job-From-Indeed
Web scraping job from Indeed using Beautifulsoup and checking top words and specific skills according to the description of the job.

In [17]:
# importing the libraries
import requests
import pandas as pd
import re
import string
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
from operator import itemgetter
from datetime import datetime

### Let´s request access to the Indeed website, looking for Data Analyst

In [18]:
def request_URL(URL):
    #show the website
    #print(URL)
    
    #conducting a request of the stated URL above:
    page = requests.get(URL)

    #specifying a desired format of 'page' using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
    soup = BeautifulSoup(page.text, 'html.parser')
    return soup

In [19]:
# we will search for job that contain the word bellow as example
word = 'data+analyst'
URL = 'https://ie.indeed.com/jobs?as_and=' + word + '&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=&radius=100&l=Dublin&fromage=any&limit=1000&sort=date&psf=advsrch&from=advancedsearch'
soup = request_URL(URL)

### Let´s get the identification of each job to check the full description of the job

In [20]:
# get the identification of each job
keys = [div.get('data-jk') for div in soup.findAll('div', attrs={'data-tn-component': 'organicJob'})]

### Let´s request the access of each specific information

In [21]:
# For each specific job(key) get the request
job_information = []
for job in keys:
    URL_job = 'https://ie.indeed.com/viewjob?jk=' + job
    soup_job = request_URL(URL_job)
   
    company = soup_job.find(name='div', attrs={'class':'icl-u-lg-mr--sm icl-u-xs-mr--xs'}).get_text()
    title = soup_job.find(name='h1', attrs={'class':'icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title'}).get_text()
    job_description = soup_job.find('div', attrs={'id': 'jobDescriptionText'}).get_text()
    date = datetime.now()    
    
    job_information.append((date, company, title, job_description, URL_job))

In [22]:
# using set_option to show all text - not truncated
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame(job_information,columns=['date','company','title','full description', 'link'])
total_comp = len(df['company'])

### Let´s create a file with the information about the jobs selected

In [23]:
file_name = 'Files/jobs_indeed_' + word + '.csv'
df.to_csv(file_name, mode='a', encoding='utf-8')

OSError: [Errno 22] Invalid argument: 'Files/jobs_indeed_"data+analyst".csv'

### Now we need to clean up the text to get rid of space, blank lines, punctuation...

In [ ]:
# cleaning using regex
def clean_text(text):
    text = re.sub(r'[\n]',' ',text) # delete \n
    text = re.sub('\s+',' ',text) # delete spaces
    text = re.sub('[%s]' % re.escape(string.punctuation),'',text) # delete all punctuation
    text = re.sub('—','',text)
    text = re.sub('’s','',text)
    text = re.sub('Â·','',text)
    text = re.sub('â€“','',text)
    text = re.sub('â€™','',text)
    text = re.sub('. . .','',text)
    text = re.sub('##','',text)
    text = re.sub('“','',text)
    text = re.sub('”','',text)
    text = re.sub('‘','',text)
    text = re.sub('’','',text)
    return text

# cleaning the column summary using regex
df['full description'] = df['full description'].apply(lambda x: clean_text(x))

### Now we will check the frequency of the words and eliminate the irrelevant words(stopwords)

In [ ]:
blob = TextBlob(str(df['full description']))

# word_counts is a dictionary, so we use items and get rid off the stop words
items = blob.word_counts.items() 

# get rid off stop words
stop_words = stopwords.words('english')
items = [item for item in items if item[0] not in stop_words]

# now we will create a df with the frequency of the words
df_word_count = pd.DataFrame(items, columns=['word','count'])
df_word_count.sort_values(by='count',ascending=False,inplace=True)
print(f'Total of words found without stopwords: {len(df_word_count)}')

### Now let´s analyze what kind of speech we find, if the word is adjetive, verb, noum... 

In [ ]:
def part_speech(text):
    tokenized = word_tokenize(text)
    classif = [pos for word,pos in pos_tag(tokenized)]
    return ' '.join(classif)
# apply the categorizing function to the word 
df_word_count['classif'] = df_word_count['word'].apply(part_speech)

In [ ]:
# checking the part of speech founded
df_word_count['classif'].unique()

In [ ]:
# Before the visualization, let´s filter some part of speech that are not relevant
df_word_count = df_word_count[~df_word_count['classif'].isin(['IN','PRP','RB','MD','CD','DT','RBR','CC','WDT','WP$'])]

### Visualization of the Top words 

In [ ]:
# plot the top20 words
df_top = df_word_count.head(20)

%matplotlib inline
plt.style.use('fivethirtyeight')
plt.figure(figsize=(20,10))
plt.title(f'Top 20 words most found on Indeed website on jobs looking for "{word}"\nTotal Jobs: {total_comp}',fontsize=20)
plt.barh(df_top['word'],df_top['count'])
plt.yticks(fontsize=18)
plt.gca().invert_yaxis()
plt.show()

### Let´s check some skills that the companies are looking for

In [ ]:
# https://www.indeed.com/career-advice/resumes-cover-letters/data-analyst-skills-resume
skills = ['python','database','sql','oracle','sybase','tableau', 'power','pentaho','excel', 'aws','git','scrum','agile','statistic','math','report','visualiz','azure','cloud','lifecycle','etl',
          'model','decision','collect','clean','present','communication','logic','detail','predictive','warehous','machine','mining']

list_skills = []
for s in skills:
    list_skills.append((s,'/'.join(df['company'][df['full description'].str.lower().str.contains(s)])))
    
df_skill = pd.DataFrame(list_skills, columns=['skill','companies'])

# insert a column with the total of companies looking for the skill
df_skill['total'] = df_skill['companies'].str.count('/')
df_skill.sort_values(by='total',ascending=False,inplace=True)
df_skill

In [ ]:
# plot the skills per company

%matplotlib inline
plt.style.use('fivethirtyeight')
plt.figure(figsize=(20,10))
plt.title(f'SKILLS wanted by Companies that are looking for "{word}"\nSource: Indeed',fontsize=20)
plt.barh(df_skill['skill'],df_skill['total'])
plt.yticks(fontsize=18)
plt.gca().invert_yaxis()
plt.show()

### Conclusion:
Observing the plots above we can see that the companies looking for Data Analyst on Indeed website, 
Excel and SQL are the most wanted technical skill and
Data, business and experience are the most wanted soft skill